In [28]:
import pandas as pd
import networkx as nw
import sys
import time

In [29]:
df = pd.read_csv(r'C:\Users\KCC\Desktop\Flight_Data.csv')
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Pegasus Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.982660,969.270667
1,Turkish Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.982660,969.270667
2,Turkish Airlines,Atatürk International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.976898,28.814600,163,Tehran,Iran,35.416100,51.152199,3305,2040.978811,3.035377,990.489405
3,Emirates,Dubai International Airport,Imam Khomeini International Airport,Dubai,United Arab Emirates,25.252800,55.364399,62,Tehran,Iran,35.416100,51.152199,3305,1199.863567,1.990514,569.931784
4,Etihad Airways,Abu Dhabi International Airport,Imam Khomeini International Airport,Abu Dhabi,United Arab Emirates,24.433001,54.651100,88,Tehran,Iran,35.416100,51.152199,3305,1266.681453,2.073517,603.340726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12845,Kenmore Air,Boeing Field King County International Airport,William R Fairchild International Airport,Seattle,United States,47.529999,-122.302002,21,Port Angeles,United States,48.120201,-123.500000,291,110.930397,0.637802,25.465199
12846,Kenmore Air,Boeing Field King County International Airport,Orcas Island Airport,Seattle,United States,47.529999,-122.302002,21,Eastsound,United States,48.708199,-122.910004,31,138.564899,0.672130,39.282450
12847,Kenmore Air,William R Fairchild International Airport,Boeing Field King County International Airport,Port Angeles,United States,48.120201,-123.500000,291,Seattle,United States,47.529999,-122.302002,21,110.930397,0.637802,25.465199
12848,Kenmore Air,Friday Harbor Airport,Boeing Field King County International Airport,Friday Harbor,United States,48.521999,-123.024002,113,Seattle,United States,47.529999,-122.302002,21,122.677708,0.652395,31.338854


In [145]:
G = nw.DiGraph()

In [157]:
G = nw.from_pandas_edgelist(df, source = 'SourceAirport', target = 'DestinationAirport', edge_attr = ['Distance','FlyTime','Price','SourceAirport_City','SourceAirport_Country','DestinationAirport_City','DestinationAirport_Country'])

In [158]:
source_and_destination_airports = input()
airports = source_and_destination_airports.split(' - ')
start_node , target_node = airports[0],airports[1]

In [159]:
def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.nodes())
    shortest_path = {}
    previous_nodes = {}
    # We'll use max_value to initialize the "infinity" value of the unvisited nodes   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # However, we initialize the starting node's value with 0   
    shortest_path[start_node] = 0
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes:  # Iterate over the nodes
            if current_min_node is None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
        # The code block below retrieves the current node's neighbors and updates their distances
        neighbors = list(graph.adj[current_min_node])
        for neighbor in neighbors:
            value = graph.get_edge_data(current_min_node, neighbor)
            tentative_value = shortest_path[current_min_node] + value.get('Distance')
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # We also update the best path to the current node
                previous_nodes[neighbor] = current_min_node
        unvisited_nodes.remove(current_min_node)
    return previous_nodes, shortest_path   

In [160]:
def dijkstra_calculate_result(previous_nodes, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]
 
    path.append(start_node)
    path.reverse()
    return path

In [161]:
def dijkstra_write_file(graph, path, target_node, shortest_path, execution_time):
    total_price = 0
    total_time = 0
    f = open("Blaster-UIAI4021-PR1-Q1.txt", "w")
    f.write("dijkstra Algorithm\n"
            "Execution Time: {}\n"
            ".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-\n".format(round(execution_time)))
    for i in range(len(path) - 1):
        value = graph.get_edge_data(path[i], path[i + 1])
        total_price = total_price + value.get('Price')
        total_time = total_time + value.get('FlyTime')
        f.write("Flight #{}:\n"
                "From: {}-{}, {}\n"
                "To: {}-{}, {}\n"
                "Duration: {}km\n"
                "Time: {}h\n"
                "Price: {}$"
                .format(i + 1, value.get('SourceAirport_City'), path[i], value.get('SourceAirport_Country'),
                        value.get('DestinationAirport_City'), path[i + 1], value.get('DestinationAirport_Country'),
                        round(value.get('Distance')),
                        round(value.get('FlyTime')),
                        round(value.get('Price'))))
        f.write("\n----------------------------\n")
    f.write("Total Price: {}$\n"
            "Total Duration: {}km\n"
            "Total Time: {}h".format(round(total_price), round(shortest_path[target_node]), round(total_time)))
    f.close()
    

In [162]:
dijkstra_start_time = time.time()

In [163]:
previous_nodes, shortest_path = dijkstra_algorithm(graph=G, start_node=start_node)

In [164]:
path = dijkstra_calculate_result(previous_nodes, start_node=start_node, target_node=target_node)

In [165]:
dijkstra_end_time = time.time()

In [166]:
dijkstra_execution_time = dijkstra_end_time - dijkstra_start_time

In [167]:
dijkstra_write_file(graph=G, path=path, target_node=target_node, shortest_path=shortest_path, execution_time= dijkstra_execution_time)